# DS 453 / 653 Spring 2024: Project 1

---

_You must follow the Academic Code of Conduct and Collaboration Policy stated in the course syllabus at all times while working on this project._

This Jupyter notebook file explains the tasks that you must complete for Project 1. You must submit your final project on [Gradescope](https://www.gradescope.com/courses/497424). You have two opportunities to submit the project:

- The first submission date is Thursday, March 21 at 8pm. You __must__ complete Parts 1-3 by then to receive credit for the project! You can complete Parts 4-5 as well, or it's okay if those are still a work in progress. We will review your work and tell you whether you have passed, or otherwise what is needed to reach a passing grade.
- The second submission date is Thursday, March 28 at 8pm. You must complete the entire project by this time.

## Project Overview

In this project, you will explore how to use the Bitcoin blockchain. More precisely, you will use an alternative blockchain called "testnet." This blockchain acts like the real Bitcoin blockchain and uses nearly-identical code, except it is just for testing purposes and the coins don't "count" as real money.

There are five parts to this assignment:

1. Receive money on Bitcoin testnet.
2. Send money to yourself on testnet.
3. Send money to your classmates on testnet.
4. Trace the transactions between your classmates on testnet.
5. Return your coins.

I imagine that the bulk of the work will be in Parts 3 and 4. Because Part 4 relies on Part 3 being completed, the first three parts must be done by the first submission date of March 21.

An important warning before we continue:
> This assignment involves creating secret and public keys for Bitcoin testnet wallets. If you happen to own any cryptocurrency, do NOT use any of those keys in this project! (And also, don't reuse any keys generated here for any other purpose after the project is over.)

In [1]:
# Execute this code block only if you are using Google Colab.
# Otherwise, run the commands locally on your own computer.

!pip install "cryptos @ git+https://github.com/nicolas3355/cryptos"
!pip install pycryptodome

  Cloning https://github.com/nicolas3355/cryptos to /private/var/folders/zk/wg0rx4b968n669r91ch1ymv40000gn/T/pip-install-iggqgjor/cryptos_2e029a4671394785870e9af1dbac6a1a
  Running command git clone --filter=blob:none --quiet https://github.com/nicolas3355/cryptos /private/var/folders/zk/wg0rx4b968n669r91ch1ymv40000gn/T/pip-install-iggqgjor/cryptos_2e029a4671394785870e9af1dbac6a1a
  Resolved https://github.com/nicolas3355/cryptos to commit 5dd9df1049a8e2ba5b2ec7cc0067a4ed8e875fd1
  Preparing metadata (setup.py) ... done


In [1]:
from cryptos.keys import PublicKey, b58decode
from cryptos.transaction import Tx, TxIn, TxOut, Script, OP_CODE_NAMES
from cryptos.ecdsa import sign, verify, Signature
from io import BytesIO
from binascii import hexlify

## Part 1: Receiving money on Bitcoin testnet

In this part of the project, you will create a cryptocurrency wallet and receive some coins.

Because real Bitcoins are expensive, in this task we will use the Bitcoin testnet instead. Testnet is a blockchain that uses all the real code of Bitcoin -- there are clients and nodes that have accounts and post transactions, and also miners who create blocks. There's just one big difference: the money doesn't count.


### Task 1.1: Create a Bitcoin testnet address

Execute the code below to generate a secret/public key pair for testnet.

> __Make sure that you only execute this block of code once!__ In fact, I recommend that you comment out this block after you have executed it, just to be sure that you do not execute it ever again. If you lose your secret key, _you won't be able to complete future steps and you will have to start over_.

In [9]:
# Execute this cell block ONLY ONCE! (and do not modify it)

from cryptos.keys import gen_secret_key, PublicKey
from cryptos.bitcoin import BITCOIN

# generate a random secret/public key pair
secret_key = gen_secret_key(BITCOIN.gen.n)
public_key = PublicKey.from_sk(secret_key)
public_key_address = public_key.address('test', False)

# print the secret key, as an integer
print("Secret key:", secret_key)

# print the address
print("Address:", public_key_address)

Secret key: 86213463247682137805449584713089544021502436286025021651775750518286425215467
Address: mmkvTuAtudCjiRZeUNMPbsfUFYjHEWj9kF


To be extra cautious that you don't forget your Bitcoin secret key, copy/paste it into the code block below. This way, if you accidentally re-run the previous code block or otherwise overwrite the `secret_key` variable, this code block will always allow you to get it back.

In [6]:
from cryptos.keys import gen_secret_key, PublicKey
from cryptos.bitcoin import BITCOIN

In [7]:
secret_key = 86213463247682137805449584713089544021502436286025021651775750518286425215467 # write your secret key here, and save the Jupyter notebook file once you have done so
public_key = PublicKey.from_sk(secret_key)
address = public_key.address('test', False)
pk_hashed = public_key.encode(False, hash160=True)

# print the address
print("Address:", address)

Address: mmkvTuAtudCjiRZeUNMPbsfUFYjHEWj9kF


Do not proceed further until you have stored your Bitcoin secret key!! You can even save a backup copy of it as a file on your computer somewhere else, if you want.

Whereas the secret key must be kept secret, the `address` is something you can share freely with anyone so that they can send you coins. Next, we'll see how that works.

### Task 1.2: Receive coins

On testnet, there are services called _faucets_ that will give you testnet-Bitcoins (written as tBTC) for free.

Here's how to receive tBTC:

1. Visit the website https://coinfaucet.eu/en/btc-testnet/.

2. Submit your testnet address to the website. The address is a hash of your public key. (Remember: _do not send your secret key_! Never send your secret key to anyone.)

3. Write down the following pieces of information from the confirmation page.

In [14]:
# Write down how many testnet-Bitcoins you received. It should be a fraction of a coin,
# data type: floating point number, written to 8 digits of precision

tBTC_received = 0.00044735 # todo: write here!

# Write down the address where you can send back the testnet coins once you are finished with them
# Most likely, you should only need to uncomment the address shown below
return_address =  "tb1qerzrlxcfu24davlur5sqmgzzgsal6wusda40er"

# converting from tBTC to Satoshis, which are the smallest unit of money within Bitcoin
# in other words, multiply by 10^8
# data type: integer
sat_received = int(tBTC_received * 100000000)

### Task 1.3: Find your coins

On testnet, a block gets mined every 10 minutes on average.  Check that a transaction has been mined (or is still in mempool waiting to be picked up by a miner) by visiting https://mempool.space/testnet/mempool-block/0 and searching for your address. Remember: only copy/paste your public address into mempool; never share your secret key with anyone!

At first you will see that the confirmation is unconfirmed but soon it will get picked up by a miner. Your address should be recorded in 1 transaction, where you received some coins from the faucet. Click on the link to explore that transaction in more detail.

Copy the transaction hash (as a hex string) below. For example if your transaction was https://mempool.space/testnet/tx/5c8288498b29d8b6208f924bafddaa5794549cb2d33e084994708c58e52d62e0 then your transaction hash would be 5c8288498b29d8b6208f924bafddaa5794549cb2d33e084994708c58e52d62e0.

In [13]:
# transaction hash, corresponding to the transaction that sent money into your address
# data type: hex string
transaction_hash = "b5a41012e939dd170827f3f9d922de2105e2571ae03a21a9b5edc73cfde3bd93" # todo: write here!

Next, click on the "details" button next to "Inputs & Outputs." Your transaction is likely to have one input account and two outputs.

- The input account is owned by the faucet.
- One of the two outputs belongs to the faucet. The other one, if you've done the above steps correctly, should belong to you!

The two output scripts both have the following format.

    OP_DUP
    OP_HASH160
    OP_PUSHBYTES_20 [hashed-public-key]
    OP_EQUALVERIFY
    OP_CHECKSIG

Run the code below to check that your public key is included in one of the locked output scripts. Your bitcoin address is just the hash of your public key, but using a hash function called `Hash160` rather than the `SHA-256` function we have discussed so far in class.

In [10]:
# Do NOT modify this code block!

from cryptos.keys import b58decode, b58encode
from cryptos.sha256 import sha256
from binascii import hexlify

print("OP_DUP")
print("OP_HASH160")
print("OP_PUSHBYTES_20 " + hexlify(pk_hashed).decode("ascii"))
print("OP_EQUALVERIFY")
print("OP_CHECKSIG")

OP_DUP
OP_HASH160
OP_PUSHBYTES_20 44750ce49c1985d33e6df98ca1ac66330e1b4415
OP_EQUALVERIFY
OP_CHECKSIG


Which output is the one corresponding to your address? Write down the index, starting at 0. That is, consider the top output to be index 0, the one after that to be index 1, and so forth.

In [15]:
# write down the index corresponding to the script that you own
# data type: integer
script_index = 0 # todo: write here!

The script above says that:

> Whoever can provide (1) a public key that hashes to the specified hash digest and (2) a valid digital signature corresponding to this public key has the right to transfer these coins somewhere else.

Because you know the secret key, you can unlock this script and spend its coins!

_Just to confirm: you did keep your secret key safe, right? Remember that you will need it for the rest of this project, because only that gives you the power to send coins. If you lose your secret key, you will have to start the project over again. You don't want that._

## Part 2: Send money to yourself

In this part of the project, you will post a transaction to the testnet blockchain to send some of your coins to the instructor, and keep the rest of the coins for yourself.

### Task 2.1: Create a transaction

In this step, you are going to send some of your tBTC coins to the instructors. Our address is "mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe"

Please read and understand the code block below, and then execute it. It will produce a script to send 100 Satoshi to the instructors (you can change this amount if you wish), 300 Satoshi as a fee to the miner who puts this transaction into the block, and the remaining money back to yourself.

In [16]:
# Execute, but do NOT modify this code block!

from cryptos.transaction import Tx, TxIn, TxOut, Script, OP_CODE_NAMES
from cryptos.ecdsa import sign, verify, Signature
from binascii import hexlify

# first, let's specify the input to the transaction
# this will tell the Bitcoin network where you received the money that you are now spending
in_pkb_hash = hexlify(pk_hashed).decode("ascii")

tx_in = TxIn(
    net = "test",                              # specifying that the network is ran on testnet
    prev_tx = bytes.fromhex(transaction_hash), # transaction in which you received the coins
    prev_index = script_index,                 # index within the transaction where you received coins
    script_sig = None,                         # this field will have the digital signature, to be inserted later
)
tx_in.rev_tx_script_pubkey = Script([118, 169, in_pkb_hash, 136, 172]) # writing the script that you are going to unlock


# transaction output 1: send money to the instructors
# this is the pk_hashed for the instructors' Bitcoin testnet wallet, corresponding to our address mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe
out1_pkb_hash = bytes.fromhex("a9102d676b93d159fff7041d4f36f78b585c54df")
out1_script = Script([118, 169, out1_pkb_hash, 136, 172]) # OP_DUP, OP_HASH160, <hash>, OP_EQUALVERIFY, OP_CHECKSIG
tx_out1 = TxOut(
    amount = 100, # sending 100 Satoshis to the instructors
    script_pubkey=out1_script
)

# transaction output 2: sending the remaining money back to you
out2_pkb_hash = pk_hashed
out2_script = Script([118, 169, out2_pkb_hash, 136, 172])
tx_out2 = TxOut(
    amount = sat_received - 100 - 300, # leaving 300 Sat unclaimed, so the miner can take it as a fee
    script_pubkey=out2_script
)

tx = Tx(
    version = 1,
    tx_ins = [tx_in],
    tx_outs = [tx_out1, tx_out2],
)

# sign the input transaction and unlock the script
message = tx.encode(sig_index = 0)
sig = sign(secret_key, message)
sig_bytes_and_type = sig.encode() + b'\x01' # DER signature + SIGHASH_ALL
pubkey_bytes = public_key.encode(compressed=False, hash160=False)
script_sig = Script([sig_bytes_and_type, pubkey_bytes])
tx_in.script_sig = script_sig

# print the transaction, in hex
print(tx.encode().hex())

010000000193bde3fd3cc7edb5a9213ae01a57e20521de22d9f9f3270817dd39e91210a4b5000000008b483045022100eb96d15866ce3639db6e81ecfa81e7d4dea9f08d7eefb67dce0885c7e8fb00ad02205d5fe2c71047d1ff0c294f1a89fe16f9e839a10e6f84dad4cebe897148bf19fd014104fe42ff6d09407c944e7f796225e04a88e47e2fb0a2b6c96b1662598eb1b92cd6b88e9a16992a4989436545a6d11e8f34ff7041c803f5719e00e337a04290fbd2ffffffff0264000000000000001976a914a9102d676b93d159fff7041d4f36f78b585c54df88ac2fad0000000000001976a91444750ce49c1985d33e6df98ca1ac66330e1b441588ac00000000



Now do the following steps:

1. Copy the transaction in hex and paste it here: https://blockstream.info/testnet/tx/push. They will broadcast your transaction to the testnet nodes, and within a few minutes one of the miners will include it in a block.

2. Write down your transaction id below, so we can find it and check that you have completed this step successfully.

__Task 2.1 response:__

[Todo: write the transaction ID here! If you can do so, please also attach a screenshot of the mempool.space page for this transaction.]

b1584eea6771ab50b358e5e4d06d3f78c88b19eb02d78d537cd1970b8b13f7c6

Screenshot:



### Task 2.2: Improve pseudonymity

The code block in Task 2.1 above suffers from one defect: it creates a transaction that shows money being spent out of an account, and most of the coins are transferred back into the _same_ account. This is not ideal from the perspective of privacy; it could allow an attacker to link transactions over time.

A better strategy is to create a new account to receive the funds. Modify the code from Task 2.1 to do this. In fact: create a function that allows you to spend some of your own money, and also keep the change in a different wallet. As before, we're going to consider a transaction with one input address and two output addresses.

_Reminder: whenever you create a new secret key, you must make sure to remember it somewhere! If you lose any secret key that you generate in this project, you will be in trouble. Make sure to store all keys safely._

__Task 2.2 response:__

In [8]:
def create_transaction(in_pkb_hash, transaction_hash, script_index, out1_pkb_hash, out1_amount, out2_pkb_hash, out2_amount):
    # TODO: write here!

    tx_in = TxIn(
        net = "test",
        prev_tx = bytes.fromhex(transaction_hash), 
        prev_index = script_index,
        script_sig = None,
    )
    tx_in.rev_tx_script_pubkey = Script([118, 169, in_pkb_hash, 136, 172])
    
    # transaction output 1: send money to the first specified address
    out1_script = Script([118, 169, out1_pkb_hash, 136, 172])
    tx_out1 = TxOut(
        amount = out1_amount,
        script_pubkey=out1_script
    )
    
    # transaction output 2: send the remaining amount to the second specified address
    out2_script = Script([118, 169, out2_pkb_hash, 136, 172])  
    tx_out2 = TxOut(
        amount = out2_amount,
        script_pubkey=out2_script
    )
    
    tx = Tx(
        version = 1,
        tx_ins = [tx_in],
        tx_outs = [tx_out1, tx_out2],
    )
    
    # sign the input transaction and unlock the script
    message = tx.encode(sig_index = 0)
    sig = sign(secret_key, message)
    sig_bytes_and_type = sig.encode() + b'\x01'
    pubkey_bytes = public_key.encode(compressed=False, hash160=False)
    script_sig = Script([sig_bytes_and_type, pubkey_bytes])
    tx_in.script_sig = script_sig
    
    return tx.encode().hex()

In [18]:
# See how its work, and hopefully it works

change_secret_key = gen_secret_key(BITCOIN.gen.n)
change_public_key = PublicKey.from_sk(change_secret_key)
change_pkb_hash = change_public_key.encode(False, True)

tx_hex = create_transaction(pk_hashed, "b5a41012e939dd170827f3f9d922de2105e2571ae03a21a9b5edc73cfde3bd93", 0, bytes.fromhex("a9102d676b93d159fff7041d4f36f78b585c54df"), 100, change_pkb_hash, 44335)
print(tx_hex)

010000000193bde3fd3cc7edb5a9213ae01a57e20521de22d9f9f3270817dd39e91210a4b5000000008a473044022001b6734b3954530a10918ff59a3ad7c19501b31e5a53ded6dea0dd0346f1d1b00220601e8ea98cbe4f11ff8edf6237593ab8e18edee02161195031351d96d191cf50014104fe42ff6d09407c944e7f796225e04a88e47e2fb0a2b6c96b1662598eb1b92cd6b88e9a16992a4989436545a6d11e8f34ff7041c803f5719e00e337a04290fbd2ffffffff0264000000000000001976a914a9102d676b93d159fff7041d4f36f78b585c54df88ac2fad0000000000001976a91411f7364553863e5ebef2f795fc6d4ada167b2cdc88ac00000000


## Part 3: Send money to your classmates

Now that you know how to send money to yourself, send some money to your classmates.

In order to complete this part of the project: find at least 3 classmates in the Crypto for Data Science class, and send some testnet coins to them.

Remember that other students will be asking you for an address so that they can send money to you, in order to complete this part of their project. Use the technique you learned in Task 2.2 to improve pseudonymity: your goal should be to give each classmate a fresh new address that cannot be linked back to the rest of your money.

Show all relevant code below.

### Task 3.1: Create many Bitcoin testnet addresses pseudorandomly

In order to try to preserve your pseudonymity, you should hand a different Bitcoin address to each of your classmates. One way that you could do this is to run the code block from Task 1.1 over and over again to generate many secret/public key pairs, and then carefully keep track of all of your secret keys. That will work, but it's tedious.

Instead, let's use the idea of _hierarchical deterministic wallets_ that we learned in Lecture 8 (...but without the hierarchy). Specifically, your goal here is to generate a method that would allow you to protect a single __secret seed__, and use that to generate many secret/public key pairs.

In the `cryptos` library, we have created a new method called `gen_secret_key_from_bytes` that will deterministically (but pseudorandomly) generate a secret key given a seed. Here is an example to show how it works.

In [1]:
def gen_secret_key_from_bytes(n: int, secret: bytes) -> int:
    """
    Generates a valid secret key for elliptic curve cryptography from a given byte sequence.

    Parameters:
    - n (int): The upper bound on the key, typically representing the order of the elliptic curve being used.
    The function ensures that the generated key is valid, i.e., 1 <= key < n.
    - secret (bytes): The initial byte sequence from which the secret key is derived. This byte sequence
    is used to generate an integer value that, if within the valid range, becomes the secret key.

    Returns:
    - int: A valid secret key derived from the provided byte sequence, where 1 <= key < n. If the initial
    conversion of `secret` to an integer does not yield a valid key, the function repeatedly hashes `secret`
    using SHA-256 until a valid key is obtained.

    Note: The function employs a while loop to continually hash the `secret` using SHA-256 and convert it
    to an integer until a valid key within the specified range is generated.
    """
    while True:
        key = int.from_bytes(secret, 'big')
        if 1 <= key < n:
            break # the key is valid, break out
        secret = sha256(secret)
    return key

In [34]:
# Execute this code block multiple times.
# Observe that the output is the same each time, unlike in Part 1.
# That's because the new method gen_secret_key_from_bytes is repeatable if you use the same seed.

from cryptos.keys import PublicKey
from cryptos.bitcoin import BITCOIN

super_secret_seed = b"1234567890123456789012345678901234567890123456789012345678901234567890"
secret_key = gen_secret_key_from_bytes(BITCOIN.gen.n, super_secret_seed)
public_key = PublicKey.from_sk(secret_key)
public_key_address = public_key.address('test', False)

# print the secret key, as an integer
print("Secret key:", secret_key)

# print the address
print("Address:", public_key_address)

Secret key: 34898150363254957465749142463706752256725619842651514335399468822696204774367
Address: myb5SsjenAiczwoi8HtRkC3UwDrGAZMTza


This method will get you part of the way toward the goal. But remember that we want to store only one secret_seed and generate __many__ secret/public key pairs from it.

Your task is to use the ideas from Lecture 8 in order to design a method that will generate at least 10 secret/public keys from a single seed.

__Your response:__

In [43]:
# Random secret seed generator

import secrets

secret_seed = secrets.token_hex(64)
secret_seed

'a2bf0fa2df02b3e7462bcb874a3a77fd0834f8e2c3972a9eae6c7a36d7b70ff78a2a6e31ec6bc5c7debeff7d4f86c202b3a9c890d2fd835a8486f3ec74dbdc59'

In [44]:
provided_string_length = len(secret_seed)
provided_string_length

128

In [42]:
# Todo: write here!

from cryptos.keys import PublicKey
from cryptos.bitcoin import BITCOIN

def generate_key_pairs(super_secret_seed, num_pairs):
    key_pairs = []
    for i in range(num_pairs):
        # Append the index to the seed to create a unique seed for each key pair
        seed = super_secret_seed + str(i).encode('utf-8')
        
        secret_key = gen_secret_key_from_bytes(BITCOIN.gen.n, seed)
        public_key = PublicKey.from_sk(secret_key)
        public_key_address = public_key.address('test', False)
        
        key_pairs.append((secret_key, public_key_address))
    
    return key_pairs

# Set the super secret seed
super_secret_seed = b"30960f836a331c5b2d55e1286ec35e1cd9cbb0cd069bf8de2d8f4953aae8339d3a72bf5b9aaa88bfa919e5c481d83f985a9e5592b71aa787018caece66c0b93e"

key_pairs = generate_key_pairs(super_secret_seed, 10)

for i, (secret_key, public_key_address) in enumerate(key_pairs):
    print(f"Key Pair {i+1}:")
    print("Secret Key:", secret_key)
    print("Public Key Address:", public_key_address)
    print()

Key Pair 1:
Secret Key: 44790095964990771138903139625999599421485523696264668359858934263594597813363
Public Key Address: mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN

Key Pair 2:
Secret Key: 72018828130814055611621206240743556230478335516946051021583954813256041695567
Public Key Address: muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL

Key Pair 3:
Secret Key: 68782841223073001781463839212706515651865519328044947059080032072840394932105
Public Key Address: mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ

Key Pair 4:
Secret Key: 27532891235425242232405191369042427338346803701350967808132609213742534165981
Public Key Address: mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb

Key Pair 5:
Secret Key: 36750819764865258227186619617124486339137252342474902480757269400378939749197
Public Key Address: mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq

Key Pair 6:
Secret Key: 107961204304364210722849804011648237431254258396046584310173139236557017089941
Public Key Address: mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF

Key Pair 7:
Secret Key: 123962676400753335048790968

### **Final List of Key Pairs**

Key Pair 1:
Secret Key: 72018828130814055611621206240743556230478335516946051021583954813256041695567
Public Key Address: muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL

Key Pair 2:
Secret Key: 68782841223073001781463839212706515651865519328044947059080032072840394932105
Public Key Address: mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ

Key Pair 3:
Secret Key: 44790095964990771138903139625999599421485523696264668359858934263594597813363
Public Key Address: mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN

Key Pair 4:
Secret Key: 27532891235425242232405191369042427338346803701350967808132609213742534165981
Public Key Address: mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb

Key Pair 5:
Secret Key: 36750819764865258227186619617124486339137252342474902480757269400378939749197
Public Key Address: mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq

Key Pair 6:
Secret Key: 107961204304364210722849804011648237431254258396046584310173139236557017089941
Public Key Address: mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF

Key Pair 7:
96826829785294408127010814745885561484154855572973349
Public Key Address: mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk

Key Pair 8:
Secret Key: 8756772895340743816181392985678431329049275009075896818502590479335813236764
Public Key Address: motL6Hw6haUuRP2m4navXdxk58VP1SPmC6

Key Pair 9:
Secret Key: 19128580437730048126415319776834941784822312213853445455598222319530292917288
Public Key Address: n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk

Key Pair 10:
Secret Key: 28356143567713851662416048962330017238900232264350083777358765220934379738357
Public Key Address: myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx

### **Public Key Hashing**

#### **Key Pair 2**

In [25]:
key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"),
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"),
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"),
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"),
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"),
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"),
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"),
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"),
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"),
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx")
]

# Key Pair 2 secret key and public key
key_pair_2_secret_key = key_pairs[1][0]
key_pair_2_public_key = PublicKey.from_sk(key_pair_2_secret_key)

key_pair_2_pkb_hash = key_pair_2_public_key.encode(compressed=False, hash160=True)

key_pair_2_pkb_hash_hex = key_pair_2_pkb_hash.hex()

print("Public Key Hash (hash160) for mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ:", key_pair_2_pkb_hash_hex)

Public Key Hash (hash160) for mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ: 6215ded8e0f84a1c1f8b9fc657c9efa2627db3b4


#### **Key Pair 3**

In [26]:
key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"),
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"),
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"),
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"),
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"),
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"),
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"),
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"),
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"),
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx")
]

# Key Pair 3 secret key and public key
key_pair_3_secret_key = key_pairs[2][0]
key_pair_3_public_key = PublicKey.from_sk(key_pair_3_secret_key)

key_pair_3_pkb_hash = key_pair_3_public_key.encode(compressed=False, hash160=True)

key_pair_3_pkb_hash_hex = key_pair_3_pkb_hash.hex()

print("Public Key Hash (hash160) for mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN:", key_pair_3_pkb_hash_hex)

Public Key Hash (hash160) for mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN: a97a9a3103bd24f869dba4a81cc7bcf0d2d8d6c8


#### **Key Pair 10**

In [66]:
key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"),
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"),
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"),
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"),
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"),
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"),
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"),
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"),
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"),
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx")
]

# Key Pair 10 secret key and public key
key_pair_10_secret_key = key_pairs[9][0]
key_pair_10_public_key = PublicKey.from_sk(key_pair_10_secret_key)

# Get the hash160 of the public key
key_pair_10_pkb_hash = key_pair_10_public_key.encode(compressed=False, hash160=True)

# Convert the hash160 to a hex string
key_pair_10_pkb_hash_hex = key_pair_10_pkb_hash.hex()

print("Public Key Hash (hash160) for myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx:", key_pair_10_pkb_hash_hex)

Public Key Hash (hash160) for myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx: c97102ff9ba7a2aaba6377da97ffa9f9a3fdcb67


## **Create Transactions**

In [3]:
def create_transaction(in_pkb_hash, transaction_hash, script_index, out1_pkb_hash, out1_amount, out2_pkb_hash, out2_amount):
    # TODO: write here!

    tx_in = TxIn(
        net = "test",
        prev_tx = bytes.fromhex(transaction_hash), 
        prev_index = script_index,
        script_sig = None,
    )
    tx_in.rev_tx_script_pubkey = Script([118, 169, in_pkb_hash, 136, 172])
    
    # transaction output 1: send money to the first specified address
    out1_script = Script([118, 169, out1_pkb_hash, 136, 172])
    tx_out1 = TxOut(
        amount = out1_amount,
        script_pubkey=out1_script
    )
    
    # transaction output 2: send the remaining amount to the second specified address
    out2_script = Script([118, 169, out2_pkb_hash, 136, 172])  
    tx_out2 = TxOut(
        amount = out2_amount,
        script_pubkey=out2_script
    )
    
    tx = Tx(
        version = 1,
        tx_ins = [tx_in],
        tx_outs = [tx_out1, tx_out2],
    )
    
    # sign the input transaction and unlock the script
    message = tx.encode(sig_index = 0)
    sig = sign(secret_key, message)
    sig_bytes_and_type = sig.encode() + b'\x01'
    pubkey_bytes = public_key.encode(compressed=False, hash160=False)
    script_sig = Script([sig_bytes_and_type, pubkey_bytes])
    tx_in.script_sig = script_sig
    
    return tx.encode().hex()

### **Test Run 1**

In [17]:
def address_to_pkb_hash(address):
    # Decode the address from base58 to bytes
    decoded_address = b58decode(address)
    # Extract the public key hash (hash160) from the decoded address
    pkb_hash = decoded_address[1:-4]
    return pkb_hash

def create_transaction(in_pkb_hash, transaction_hash, script_index, out1_pkb_hash, out1_amount, out2_pkb_hash, out2_amount, secret_key, public_key):
    # specify the input to the transaction
    tx_in = TxIn(
        net = "test",
        prev_tx = bytes.fromhex(transaction_hash), 
        prev_index = script_index,
        script_sig = None,
    )
    tx_in.rev_tx_script_pubkey = Script([118, 169, in_pkb_hash, 136, 172])
    
    # transaction output 1: send money to the first specified address
    out1_script = Script([118, 169, out1_pkb_hash, 136, 172])
    tx_out1 = TxOut(
        amount = out1_amount,
        script_pubkey=out1_script
    )
    
    # transaction output 2: send the remaining amount to the second specified address
    out2_script = Script([118, 169, out2_pkb_hash, 136, 172])  
    tx_out2 = TxOut(
        amount = out2_amount,
        script_pubkey=out2_script
    )
    
    tx = Tx(
        version = 1,
        tx_ins = [tx_in],
        tx_outs = [tx_out1, tx_out2],
    )
    
    # sign the input transaction and unlock the script
    message = tx.encode(sig_index = 0)
    sig = sign(secret_key, message)
    sig_bytes_and_type = sig.encode() + b'\x01'
    pubkey_bytes = public_key.encode(compressed=False, hash160=False)
    script_sig = Script([sig_bytes_and_type, pubkey_bytes])
    tx_in.script_sig = script_sig
    
    return tx.encode().hex()

# The generated key pairs (updated order)
key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"),
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"),
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"),
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"),
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"),
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"),
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"),
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"),
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"),
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx")
]

# Current address with the balance
my_address = "mmkvTuAtudCjiRZeUNMPbsfUFYjHEWj9kF"

# Secret key and public key
my_secret_key = 86213463247682137805449584713089544021502436286025021651775750518286425215467
my_public_key = PublicKey.from_sk(my_secret_key)

# The new transaction hash and script index
new_transaction_hash = "b1584eea6771ab50b358e5e4d06d3f78c88b19eb02d78d537cd1970b8b13f7c6"
new_script_index = 1

# The new amount of satoshis that I have
new_sat_received = 44335

# The amount I want to send to Key Pair 4 (in satoshis)
amount_to_send_key_pair_4 = 1000

# Get the hashed public key of my current address
my_pkb_hash = address_to_pkb_hash(my_address)

# Key Pair 4 address
key_pair_4_address = "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"
key_pair_4_pkb_hash = address_to_pkb_hash(key_pair_4_address)

# Key Pair 1 address
key_pair_1_address = "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"
key_pair_1_pkb_hash = address_to_pkb_hash(key_pair_1_address)

# Create the transaction
transaction_hex = create_transaction(
    in_pkb_hash=my_pkb_hash,
    transaction_hash=new_transaction_hash,
    script_index=new_script_index,
    out1_pkb_hash=key_pair_4_pkb_hash,
    out1_amount=amount_to_send_key_pair_4,
    out2_pkb_hash=key_pair_1_pkb_hash,
    out2_amount=new_sat_received - amount_to_send_key_pair_4 - 300,
    secret_key=my_secret_key,
    public_key=my_public_key
)

print("Transaction:", transaction_hex)

Transaction: 0100000001c6f7138b0b97d17c538dd702eb198bc8783f6dd0e4e558b350ab7167ea4e58b1010000008b483045022100f2a52b9d41eb7a6f78ffb4c910cd2858b99b68692898f536bd07b7ce8680eb0902203dc85212e330fa0ca2d208caf83df7cff9712fcd4d118774896fc02746947f2a014104fe42ff6d09407c944e7f796225e04a88e47e2fb0a2b6c96b1662598eb1b92cd6b88e9a16992a4989436545a6d11e8f34ff7041c803f5719e00e337a04290fbd2ffffffff02e8030000000000001976a914664e80c34ce3e4bd2ce3237e0beb62b4b7a24dd288ac1ba80000000000001976a9149e252972b9be85daca5f83443b9284bcad6204cd88ac00000000


### **To Jasmine, jazzmine@bu.edu**

In [54]:
def address_to_pkb_hash(address):
    # Decode the address from base58 to bytes
    decoded_address = b58decode(address)
    # Extract the public key hash (hash160) from the decoded address
    pkb_hash = decoded_address[1:-4]
    return pkb_hash

# The generated key pairs (updated order)
key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"),
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"),
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"),
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"),
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"),
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"),
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"),
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"),
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"),
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx")
]

# The address with the balance (Key Pair 1)
input_address = "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"

# The secret key and public key corresponding to the input address
secret_key = key_pairs[0][0]
public_key = PublicKey.from_sk(secret_key)

# The transaction hash and script index where I received the tBTC
transaction_hash = "9f2f41d9111f93b54f8eb75a2326dfe240bbcae6c03f810013cb25ef4a96d4b6"
script_index = 1

# The amount of satoshis I have (0.00044035 tBTC)
sat_received = 43035

# The amount that send to Jasmine (in satoshis)
amount_to_send_jasmine = 100

input_pkb_hash = address_to_pkb_hash(input_address)

# Jasmine's public key hash
jasmine_pkb_hash = bytes.fromhex("c62b6e91d58de92ad49eae6ba3b06d4dd1fac46a")

# Choose a key pair to receive the change (Key Pair 2)
change_key_pair = key_pairs[1]
change_pkb_hash = PublicKey.from_sk(change_key_pair[0]).encode(compressed=False, hash160=True)

# Create the transaction
transaction_hex = create_transaction(
    in_pkb_hash=input_pkb_hash,
    transaction_hash=transaction_hash,
    script_index=script_index,
    out1_pkb_hash=jasmine_pkb_hash,
    out1_amount=amount_to_send_jasmine,
    out2_pkb_hash=change_pkb_hash,
    out2_amount=sat_received - amount_to_send_jasmine - 300,
    secret_key=secret_key,
    public_key=public_key
)

print("Transaction:", transaction_hex)

Transaction: 0100000001b6d4964aef25cb1300813fc0e6cabb40e2df26235ab78e4fb5931f11d9412f9f010000008a47304402206583e6d1c2315034d634f36f48d7ad76a1c8ac1c5d359477a69bb87ff8325d0d02206fdf468c31656571302f4bf60c703877b2cbeddebf80a3da179cf10e501e41d3014104918a8fc3449e32b914ec9ce840979cc256b0b69304f211c48864d19d373353ad5c2b157d6cbb54312121b2c55ddbb1e554696befe07e03c6f1e1bfe8807d02c3ffffffff0264000000000000001976a914c62b6e91d58de92ad49eae6ba3b06d4dd1fac46a88ac8ba60000000000001976a9146215ded8e0f84a1c1f8b9fc657c9efa2627db3b488ac00000000


### **To Carmen, pelayo@bu.edu**

In [67]:
def address_to_pkb_hash(address):
    # Decode the address from base58 to bytes
    decoded_address = b58decode(address)
    # Extract the public key hash (hash160) from the decoded address
    pkb_hash = decoded_address[1:-4]
    return pkb_hash

# The generated key pairs
key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"), # 0
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"), # 1
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"), # 2
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"), # 3
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"), # 4
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"), # 5
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"), # 6
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"), # 7
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"), # 8
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx") # 9
]

# The address with the balance
input_address = "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"

# The secret key and public key corresponding to the input address
secret_key = key_pairs[6][0]
public_key = PublicKey.from_sk(secret_key)

# The transaction hash and script index where I received the tBTC
transaction_hash = "49b41f05cacd3654e53460b830486d6479488bff1a89d2a7a5a3d77eadae5253"
script_index = 1

# The amount of satoshis that I have
sat_received = 41785

# The amount I want to send to Hanfei
amount_to_send_carmen = 50

# Get the hashed public key of the input address
input_pkb_hash = address_to_pkb_hash(input_address)

# Carmen's public key hash
carmen_pkb_hash = bytes.fromhex("cb15cf05da8276fc7906a63381fb56ef31befbff")

# Choose a key pair to receive the change
change_key_pair = key_pairs[7]
change_pkb_hash = PublicKey.from_sk(change_key_pair[0]).encode(compressed=False, hash160=True)

# Create the transaction
transaction_hex = create_transaction(
    in_pkb_hash=input_pkb_hash,
    transaction_hash=transaction_hash,
    script_index=script_index,
    out1_pkb_hash=carmen_pkb_hash,
    out1_amount=amount_to_send_carmen,
    out2_pkb_hash=change_pkb_hash,
    out2_amount=sat_received - amount_to_send_carmen - 350,
    secret_key=secret_key,
    public_key=public_key
)

print(transaction_hex)

01000000015352aead7ed7a3a5a7d2891aff8b4879646d4830b86034e55436cdca051fb449010000008b483045022100ac8d4adee03364e8a2f73a9d47ee09814f67a6b01156de1fd4c4ba398c085fe302205cfd3f3aed69b48a89e8f8052e8d772ead036343302b6c3d2663ed8268b11003014104f767d70ef55cf9dee0794fae70fe85222bddc5544ab06759a02b72c76af620472904a347052af1bdaada9a632db97150a393a3e6ffb04a638baa09b72d7842aaffffffff0232000000000000001976a914cb15cf05da8276fc7906a63381fb56ef31befbff88aca9a10000000000001976a9145bcbefd6bdb67e4e3c40f5939eabf121e741039788ac00000000


### **To Hanfei Qi, qihanfei@bu.edu**

In [64]:
def address_to_pkb_hash(address):
    # Decode the address from base58 to bytes
    decoded_address = b58decode(address)
    # Extract the public key hash (hash160) from the decoded address
    pkb_hash = decoded_address[1:-4]
    return pkb_hash

# The generated key pairs
key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"), # 0
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"), # 1
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"), # 2
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"), # 3
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"), # 4
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"), # 5
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"), # 6
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"), # 7
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"), # 8
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx") # 9
]

# The address with the balance
input_address = "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"

# The secret key and public key corresponding to the input address
secret_key = key_pairs[4][0]
public_key = PublicKey.from_sk(secret_key)

# The transaction hash and script index where you received the tBTC
transaction_hash = "e8473dfd12fbcb6dfeff68619d126d0ad34c6307f066a85cd4fa691f7afa229a"
script_index = 1

# The amount of satoshis that I have
sat_received = 42185

# The amount you want to send to Hanfei, in satoshis
amount_to_send_hanfei = 50

# Get the hashed public key of the input address
input_pkb_hash = address_to_pkb_hash(input_address)

# Hanfei's public key hash
hanfei_pkb_hash = bytes.fromhex("2cf62619ccde35238dde9cd91d95af0e9fdb2700")

# Choose a key pair to receive the change
change_key_pair = key_pairs[6]
change_pkb_hash = PublicKey.from_sk(change_key_pair[0]).encode(compressed=False, hash160=True)

# Create the transaction
transaction_hex = create_transaction(
    in_pkb_hash=input_pkb_hash,
    transaction_hash=transaction_hash,
    script_index=script_index,
    out1_pkb_hash=hanfei_pkb_hash,
    out1_amount=amount_to_send_hanfei,
    out2_pkb_hash=change_pkb_hash,
    out2_amount=sat_received - amount_to_send_hanfei - 350,
    secret_key=secret_key,
    public_key=public_key
)

print(transaction_hex)

01000000019a22fa7a1f69fad45ca866f007634cd30a6d129d6168fffe6dcbfb12fd3d47e8010000008b483045022100d136b55f35dc7007655c32556c9fc203a65ae6d071d79f52c2c33ddeb8ea41ad0220372d350f2cbdcd92f734d51c376d05c5d651cc99cecba0bc63b7ddf1c8eafda30141045bcd0d786fa3824a8af68f96210a226a77f87f28c945412cb9a887fe207cd469085a63333741b66abfc091f97d7f7fdcf79e5350dab6bc7f3137f99a9e5f5feeffffffff0232000000000000001976a9142cf62619ccde35238dde9cd91d95af0e9fdb270088ac39a30000000000001976a914717d235138b806bdbb0781f043ed36e1ecb9318688ac00000000


----

### Task 3.2: Send and receive testnet Bitcoins

You have the two crucial ingredients that you need to send and receive tBTC.

1. You have some coins.
2. You have the ability to generate new keys/addresses on demand.

In this task, you must talk with your classmates, exchange public keys (not secret keys!), and use it to transfer tBTC through the testnet blockchain.

In the space below, provide the mempool.space links to all transactions in which you were the _sender_. There must be at least 3 such transactions. Make sure your response clearly shows the (1) Bitcoin address and (2) transaction IDs of these transactions.

__Task 3.2: transactions in which you were the sender__

---
- "Bitcoin Address: muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL",
- "Transaction ID: 800fdc065fd68f24c4b7fef896825eecae63b6387845bff98a8d34bd53878a3e",
- "Link: https://mempool.space/testnet/tx/800fdc065fd68f24c4b7fef896825eecae63b6387845bff98a8d34bd53878a3e",
- "Receiver: Jasmine",
---
- "Bitcoin Address: mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq", 
- "Transaction ID: 49b41f05cacd3654e53460b830486d6479488bff1a89d2a7a5a3d77eadae5253",
- "Link: https://mempool.space/testnet/tx/49b41f05cacd3654e53460b830486d6479488bff1a89d2a7a5a3d77eadae5253",
- “Receiver: Hanfei",
---
- "Bitcoin Address: mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk",
- "Transaction ID: 208fd22474e4b8c562661e479d9957e5f4197728add83e25232aac2c13669d7b",
- "Link: https://mempool.space/testnet/tx/208fd22474e4b8c562661e479d9957e5f4197728add83e25232aac2c13669d7b"
---

Next, provide the mempool.space links for all transactions in which you were the _receiver_. Once again, make sure your response clearly shows the (1) Bitcoin address and (2) transaction IDs of these transactions.

__Task 3.2: transactions in which you were the receiver__

---
- "Bitcoin Address: [mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN]",
- "Transaction ID: [93f15ab7b46ed2c47ff042876ef57c107b304921871d88aad6eeb675b3cb007e]", 
- "Link: https://mempool.space/testnet/tx/93f15ab7b46ed2c47ff042876ef57c107b304921871d88aad6eeb675b3cb007e",
- "Sender: Hanfei",
---
- "Bitcoin Address: mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ",
- "Transaction ID: 40897c02bb39690b9357e60638239274f06901c22a5d5aaac42947f1d528b5a9",
- "Link: https://mempool.space/testnet/tx/40897c02bb39690b9357e60638239274f06901c22a5d5aaac42947f1d528b5a9",
- "Sender: Jasmine",
---
- "Bitcoin Address: myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx", 
- "Transaction ID: bc583fabcb89d3243ea52b4f9b5eab4a2dde6cc735fa614381867274a2200b74",
- "Link: https://mempool.space/testnet/tx/bc583fabcb89d3243ea52b4f9b5eab4a2dde6cc735fa614381867274a2200b74"
- "Sender: Carmen",
---

## Part 4: Track your classmates' spending

Remember how Bitcoin works:
- you know the addresses of anyone you've ever sent money to or received money from, and
- every transaction is posted publicly for the world to see.

You can use these facts to track other people's spending! Let's see how.

### Task 4.1: Learn how to link transactions

Even though we tried in Task 2.2 to provide pseudonymity, one can make an educated guess as to which addresses belong to the same people. For example, take a look at this transaction: https://mempool.space/testnet/tx/ac44742345a22d88580f34f3a20683da955c8a1206e5b69330c61e7f8520ecf4. It was created by one of the students who performed Task 2.2 in last year's class. They sent 100 satoshis to the instructors, and from the workflow you can probably guess that the change address `mqfwaJbkuJSmrPb3oqjx2ZsQDFk1Fz1t8e` and the sender address `mm5mG34BBkLvKUfWzdQUQTpGfs5nrz2vJT` belong to the same person!

Let's read about how that can be done at scale across the Blockchain. Read [this Wired article](https://www.wired.com/story/27-year-old-codebreaker-busted-myth-bitcoins-anonymity/) about Sarah Meiklejohn's work to trace transactions across Bitcoin, such as coins sent to currency exchanges or websites that sold (sometimes illegal) goods via Bitcoin. Alternatively, you can read the [original paper](https://cseweb.ucsd.edu/~smeiklejohn/files/imc13.pdf) or [Sarah's PhD thesis](https://smeiklej.com/files/dissertation.pdf) if you prefer.

Write a summary of the reading below. We are looking for 1-2 paragraphs that show that you understood the material.

__Task 4.1 response:__

The key points behind their approach was that even though Bitcoin addresses are pseudonymous, the blockchain provides a complete, public record of every transaction. By analyzing patterns in the transaction graph, they could cluster addresses that likely belong to the same entity.
The two main heuristics they used were:

- If multiple addresses are used as inputs to the same transaction, they are likely controlled by the same user. This is because the transaction must be signed with the private keys corresponding to each input address.
- Many Bitcoin clients generate a new "change address" for each transaction to receive any leftover coins. By identifying these one-time change addresses, the researchers could link them back to the original user.

The authors engaged in transactions with various Bitcoin services and observed the addresses used, allowing them to tag clusters belonging to major entities. Combining these two heuristics with their ground truth data, they were able to cluster addresses and trace the flow of bitcoins across hundreds of thousands of transactions.

Some of their major findings included being able to track bitcoins as they were stolen from exchanges, follow the money trail of the Silk Road marketplace even as it attempted to hide its activities, and identify the major players in the Bitcoin economy. Their work demonstrates that with some effort, a significant amount of information can be gleaned from Bitcoin's public ledger, casting doubt on its anonymity guarantees in practice. This has major implications for law enforcement's ability to track illicit activity.


### Task 4.2: Track your classmates

Using your newfound knowledge, try to find as many Bitcoin addresses and transaction ids produced by your classmates! Remember that they all performed Parts 1-3 of the project as well.

Include below any code, links, screenshots, or any other information needed for us to understand (a) how you performed your linking attack and (b) what you have discovered.

To pass this part of the project, you must be able to uncover at least one classmate's address and one transction between a pair of classmates that does not involve you, and you must be able to explain how you deduced that this was a transaction involving your classmates.

__Task 4.2 response:__

My approach:

To identify transactions between classmates, I focused on the heuristic that addresses used as inputs to the same transaction are likely controlled by the same entity. I searched for transactions with multiple inputs, where one input address matched a known classmate address from my own transactions. I then assumed the other input addresses belong to the same classmate.

Another thing is, I also looked one step forward and backward in the transaction graph from known classmate addresses. For any transaction sending coins to a classmate, I assumed the change address belongs to the sender. And for transactions sent by classmates, I assumed the non-classmate output address was a mixing address, and looked for transactions onward from there.

So here is first try to implement this basic idea:

After a few rounds of the process I described above, I was able to tag several new addresses as likely belonging to our classmates.

**Hanfei**

- Addresses:
    - mjcguDS81aWNfHBaLjb7mbM5BKrJuQz2U6
    - n2HpBEudSzMzLvPzuWNsitvds8ad39GonH
    - miWjfZrPXtJgw5dCZwt7QteXXVisz2B76L

Hanfei first send me his public key address, mjcguDS81aWNfHBaLjb7mbM5BKrJuQz2U6, to me to receive the satoshis that I will send to him, which I used my address, mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq, to send to him, 50 satoshis, with the transaction hash, 49b41f05cacd3654e53460b830486d6479488bff1a89d2a7a5a3d77eadae5253. Therefore, the public key address, mjcguDS81aWNfHBaLjb7mbM5BKrJuQz2U6, must belongs to Hanfei.

Now, based on the research I did about the address, mjcguDS81aWNfHBaLjb7mbM5BKrJuQz2U6, it have three transactions, one of it must be the transaction that I send to him the satoshis, so we look at the other two. 

The transaction hash, 93f15ab7b46ed2c47ff042876ef57c107b304921871d88aad6eeb675b3cb007e, involving one of my address, mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN, as the receiver of 100 satoshis, and the remaining balance is back to the address, mjcguDS81aWNfHBaLjb7mbM5BKrJuQz2U6, which is belongs to Hanfei. Therefore this would be the transaction that Hanfei send me the money.

We have one transaction left with this address, ab66cb3ef33580fc23bd7172b640eb4d0ab9c1fd9068b29af4a68864851be830, that Hanfei is the sender, with the address, mjcguDS81aWNfHBaLjb7mbM5BKrJuQz2U6, and one of the address as the receiver, received 100 satoshis. The remaining balance is back to a new address, miWjfZrPXtJgw5dCZwt7QteXXVisz2B76L. Therefore, there is a really high chance that this transaction could be the transaction that Hanfei send to another classmate, that it is a transaction between Hanfei and another classmate that does not involve me, since I found a matching pattern of transactions, suggesting a similar purpose and context, and the creation of a new change address, which is common when a user wants to separate funds used for different purposes. 


## Part 5: Return the remaining coins

Once you are done with the project, send any remaining tBTC coins back to the original faucet so that they can be used by someone else later. Use the `return_address` that you saved in Task 1.2 to remember where the coins need to go.

I returned the tBTC coins back to the intructor's address: **mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe**, provided in the Task 2.

In [ ]:
def address_to_pkb_hash(address):
    # Decode the address from base58 to bytes
    decoded_address = b58decode(address)
    # Extract the public key hash (hash160) from the decoded address
    pkb_hash = decoded_address[1:-4]
    return pkb_hash

key_pairs = [
    (72018828130814055611621206240743556230478335516946051021583954813256041695567, "muw9aRBfXQhR5kTrRi1FWyye7MkgA5H6DL"), # 0
    (68782841223073001781463839212706515651865519328044947059080032072840394932105, "mpTaiSbgVA5MQ59uDUSuw55ZtafmWH5txQ"), # 1
    (44790095964990771138903139625999599421485523696264668359858934263594597813363, "mvy5N2mJqeX7V45EaXEQFUF8582DD7UsAN"), # 2
    (27532891235425242232405191369042427338346803701350967808132609213742534165981, "mpquG6x6SCZwjGh3kfZkzSroAKpk9hB1pb"), # 3
    (36750819764865258227186619617124486339137252342474902480757269400378939749197, "mvihM4n2boe74hkUswrUvyU6yq5rNvmyqq"), # 4
    (107961204304364210722849804011648237431254258396046584310173139236557017089941, "mzgDZd7UkYgJo6sKTWe7torBSgm7xEHXfF"), # 5
    (12396267640075333504879096826829785294408127010814745885561484154855572973349, "mqs2ZU8gD8kcQCNNMBcobmcDKtUmQgAyjk"), # 6
    (8756772895340743816181392985678431329049275009075896818502590479335813236764, "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"), # 7
    (19128580437730048126415319776834941784822312213853445455598222319530292917288, "n4aRXYJZU9jFuWyNRsKnYvhUDomTH6QmHk"), # 8
    (28356143567713851662416048962330017238900232264350083777358765220934379738357, "myt5VoYt17K7knfvMxrtWnL2u8UYEfUsUx") # 9
]

# The address with the balance
input_address = "motL6Hw6haUuRP2m4navXdxk58VP1SPmC6"

# The secret key and public key corresponding to the input address
secret_key = key_pairs[7][0]
public_key = PublicKey.from_sk(secret_key)

# The transaction hash and script index where Key Pair 8 received the tBTC
transaction_hash = "208fd22474e4b8c562661e479d9957e5f4197728add83e25232aac2c13669d7b"
script_index = 1

# The amount of satoshis that Key Pair 8 has
sat_received = 41385

# Get the hashed public key of the input address
input_pkb_hash = address_to_pkb_hash(input_address)

# The professor's address
professor_address = "mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe"
professor_pkb_hash = address_to_pkb_hash(professor_address)

# Create the transaction
transaction_hex = create_transaction(
    in_pkb_hash=input_pkb_hash,
    transaction_hash=transaction_hash,
    script_index=script_index,
    out1_pkb_hash=professor_pkb_hash,
    out1_amount=sat_received - 350, # Send all the balance minus the transaction fee
    out2_pkb_hash=input_pkb_hash,
    out2_amount=0 # No change output
)

# Sign the transaction
transaction_bytes = bytes.fromhex(transaction_hex)
message = Tx.decode(BytesIO(transaction_bytes)).encode(sig_index=0)
sig = sign(secret_key, message)
sig_bytes_and_type = sig.encode() + b'\x01'
pubkey_bytes = public_key.encode(compressed=False, hash160=False)
script_sig = Script([sig_bytes_and_type, pubkey_bytes])

tx = Tx.decode(BytesIO(transaction_bytes))
tx.tx_ins[0].script_sig = script_sig
signed_transaction_hex = tx.encode().hex()

print(signed_transaction_hex)

---
- "Sender Address: motL6Hw6haUuRP2m4navXdxk58VP1SPmC6",
- "Receiver Address: mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe",
- "Transaction ID: f83a2e04edd7bb1e5bb8e877741cb46c8dc6f96c1850666ec3f53bf6b2b37ca7",
- "Link: https://mempool.space/testnet/tx/f83a2e04edd7bb1e5bb8e877741cb46c8dc6f96c1850666ec3f53bf6b2b37ca7",
- "Receiver: Prof. Varia",
---

## Documenting collaborators, sources, and AI tools

In accordance with the collaboration policy, use the space below to report if you used any resources to complete this homework assignment, aside from the lecture notes and the course textbooks/videos. Specifically, please report:

1. Names of all classmates you worked with, and a short description of the work that you performed together.
2. All written materials that you used, such as books or websites (besides the lecture notes or textbooks). Please include links to any web-based resources, or citations to any physical works.
3. All code that you used from other sources. In particular, if you used an AI tool, then you must include the entire exchange with the AI tool, as per the [CDS Generative AI Assistance Policy](https://www.bu.edu/cds-faculty/culture-community/gaia-policy/).

Remember that if we discover any undocumented collaborators, sources, or AI tools then this is grounds for a Not Passed grade on the project without the ability to resubmit, and possible referral to BU's Academic Conduct Committee (as described in the syllabus). It is in your own interest to document all collaborators, sources, and tools used!

__Your response:__

1. People (they are really awesome!) that I worked with for this project:

- Jasmine Pham
- Carmen Pelayo Fernandez
- Hanfei Qi

2. Written Materials:

Books:

- "Mastering Bitcoin: Programming the Open Blockchain" by Andreas M. Antonopoulos (O'Reilly Media, 2017)
- "Bitcoin and Cryptocurrency Technologies: A Comprehensive Introduction" by Arvind Narayanan, Joseph Bonneau, Edward Felten, Andrew Miller, Steven Goldfeder (Princeton University Press, 2016)

Websites:

- A from-scratch tour of Bitcoin in Python, Andrej Karpathy blog: http://karpathy.github.io/2021/06/21/blockchain/
- Bitcoin Developer Documentation: https://developer.bitcoin.org/
- Bitcoin Wiki: https://en.bitcoin.it/wiki/Main_Page

Code and Libraries:

- Python-bitcoinlib: https://github.com/petertodd/python-bitcoinlib
- `cryptos` Library: https://github.com/karpathy/cryptos
- binascii.hexlify Function:
    - Documentation: https://docs.python.org/3/library/binascii.html#binascii.hexlify
- PyCryptodome’s Documentation: https://www.pycryptodome.org

1. Not applicable

When you have completed the project, please submit this .ipynb file to Gradescope.